<a href="https://colab.research.google.com/github/SankarSivan/Stock-Price-Prediction-Apple/blob/main/Stock_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


In [ ]:
import sklearn

In [ ]:
# prompt: import data from github

url = 'https://raw.githubusercontent.com/SankarSivan/Stock-Price-Prediction-Apple/main/AAPL.csv'
df = pd.read_csv(url)
print(df.head())

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
# Check for missing values
print(df.isnull().sum())

In [ ]:
# Forward fill missing values
df.fillna(method='ffill', inplace=True)

In [ ]:
# Ensure no remaining nulls
print(df.isnull().sum())

In [ ]:
df.duplicated()

In [ ]:
df.info()

In [ ]:
from datetime import date
df['Date'] = df['Date'].apply(pd.to_datetime)

In [ ]:
df.columns

In [ ]:
df.drop(columns= ['Open', 'High', 'Low', 'Adj Close', 'Volume'], inplace =True)

In [ ]:
df.info()

In [ ]:
df['Close'].plot(figsize=(12,6))

In [ ]:
pip install tensorflow

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Normalize the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[['Close']])

In [ ]:
# Create sequences
def create_sequences(data, seq_len=60, pred_gap=1):
    X, y = [], []
    for i in range(len(data) - seq_len - pred_gap + 1):
        X.append(data[i:i+seq_len])
        y.append(data[i+seq_len+pred_gap-1])  # Predicting gap days ahead
    return np.array(X), np.array(y)

In [ ]:
# Example: For 1, 5, 10 day forecasts
X_1, y_1 = create_sequences(scaled_data, pred_gap=1)
X_5, y_5 = create_sequences(scaled_data, pred_gap=5)
X_10, y_10 = create_sequences(scaled_data, pred_gap=10)

In [ ]:
# Train-test split
X, y = create_sequences(scaled_data, pred_gap=1)
split = int(0.75 * len(X_1))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

## Simple RNN Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

In [ ]:
def build_rnn_model(input_shape):
    model = Sequential()
    model.add(SimpleRNN(units=50, activation='relu', input_shape=input_shape))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

In [ ]:
rnn_model = build_rnn_model((X_train.shape[1], X_train.shape[2]))
rnn_model.fit(X_train, y_train, epochs=20, batch_size=64, validation_split=0.2)

In [ ]:
rnn_model.summary()

## LSTM Model

In [ ]:
pip install scikeras

In [ ]:
from tensorflow.keras.layers import LSTM, Dense, Dropout
from scikeras.wrappers import KerasRegressor
from tensorflow.keras.optimizers import Adam

In [ ]:
# model building
def build_lstm_model(input_shape, units = 50, Dropout_rate = 0.2, learning_rate = 0.001):
    model = Sequential()
    model.add(LSTM(units = units, return_sequences = False, input_shape = input_shape))
    model.add(Dropout(Dropout_rate))
    model.add(Dense(1))
    model.compile ( optimizer = Adam(learning_rate = learning_rate), loss = 'mse')
    return model

# Model Excution
lstm_model = build_lstm_model((X_train.shape[1], X_train.shape[2]))
lstm_model.fit(X_train, y_train, epochs = 10, batch_size = 64, validation_split =0.2)

In [ ]:
lstm_model.summary()

## Evaluate Models

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
def evaluate(model, X_test, y_test, scaler):
    pred_scaled = model.predict(X_test)
    pred = scaler.inverse_transform(np.concatenate([pred_scaled, np.zeros((len(pred_scaled), 1))], axis=1))[:,0]
    true = scaler.inverse_transform(np.concatenate([y_test.reshape(-1,1), np.zeros((len(y_test), 1))], axis=1))[:,0]
    rmse = np.sqrt(mean_squared_error(true, pred))
    mae = mean_absolute_error(true, pred)
    return rmse, mae

rmse_rnn, mae_rnn = evaluate(rnn_model, X_test, y_test, scaler)
rmse_lstm, mae_lstm = evaluate(lstm_model, X_test, y_test, scaler)

print(f"SimpleRNN RMSE: {rmse_rnn:.2f}, MAE: {mae_rnn:.2f}")
print(f"LSTM RMSE: {rmse_lstm:.2f}, MAE: {mae_lstm:.2f}")

In [ ]:
# predict and inverse scale
y_pred = lstm_model.predict(X_test)
y_pred_rescaled = scaler.inverse_transform(np.concatenate([y_pred, np.zeros((len(y_pred),df.shape[1]-1))], axis = 1))[:,0]
y_test_rescaled = scaler.inverse_transform(np.concatenate([y_test.reshape(-1,1), np.zeros((len(y_test), df.shape[1]-1))], axis = 1)) [:,0]


In [ ]:
# Model Evaluation
results = []

In [ ]:
from keras import backend as K

In [ ]:
# plot the Actual vs Predicted
plt.figure(figsize = (12,5) )
plt.plot(y_test_rescaled, label = 'Actual')
plt.plot(y_pred_rescaled, label = 'Predicted')
plt.legend()
plt.title(' Stock Price Prediction')

# Hyperparameter Tuning with GridSearchCV

In [ ]:
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
def build_model(units=50, dropout=0.2, learning_rate=0.001):
    model = Sequential()
    model.add(LSTM(units, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(1))
    model.compile(loss="mse", optimizer=Adam(learning_rate=learning_rate))
    return model

In [ ]:
reg = KerasRegressor(model=build_model, verbose=0, epochs=10, batch_size=32)

In [ ]:
param_grid = {
    "model__units": [32, 50, 64],
    "model__learning_rate": [0.001, 0.005],
    "batch_size": [32],
    "epochs": [10, 20]
}

grid = GridSearchCV(reg, param_grid, scoring="neg_mean_squared_error", cv=3, verbose=2)
grid.fit(X_train, y_train)

print("Best params:", grid.best_params_)

In [ ]:
import numpy as np, pandas as pd, tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from itertools import product

In [ ]:
import tensorflow as tf

In [ ]:
# --- Load data ---
df = pd.read_csv(url, parse_dates=["Date"], index_col="Date")[["Close"]]
df.fillna(method="ffill", inplace=True)


# --- Scale data 0‑1 ---
scaler = MinMaxScaler()
scaled = scaler.fit_transform(df)


# --- Build 60‑step sequences to predict next‑day price ---
SEQ_LEN, GAP = 60, 1
X, y = [], []
for i in range(len(scaled) - SEQ_LEN - GAP + 1):
    X.append(scaled[i:i+SEQ_LEN])
    y.append(scaled[i+SEQ_LEN+GAP-1])
X, y = np.array(X), np.array(y)

# Train / validation split
split = int(0.8 * len(X))
X_train, X_val = X[:split], X[split:]
y_train, y_val = y[:split], y[split:]

In [ ]:



param_grid = {
    "units":        [32, 64],
    "learning_rate": [0.001, 0.005],
    "epochs":       [10, 20]
}
search_space = list(product(*param_grid.values()))
print(f"{len(search_space)} combinations to test")

In [ ]:
def build_lstm(units, lr):
    model = Sequential([
        LSTM(units, input_shape=(SEQ_LEN, 1)),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(lr), loss="mse")
    return model

results = []
best_rmse = np.inf
best_model = None

for units, lr, n_epochs in search_space:
    print(f"Training: units={units}, lr={lr}, epochs={n_epochs}")
    model = build_lstm(units, lr)
    es = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
    model.fit(X_train, y_train,
              epochs=n_epochs, batch_size=32,
              validation_data=(X_val, y_val),
              callbacks=[es], verbose=0)

    # Validate
    val_pred = model.predict(X_val, verbose=0)
    val_rmse = np.sqrt(mean_squared_error(y_val, val_pred))
    results.append((units, lr, n_epochs, val_rmse))

    # Keep best
    if val_rmse < best_rmse:
        best_rmse, best_model = val_rmse, model
        best_params = (units, lr, n_epochs)
        print(f"  🔥 New best RMSE: {best_rmse:.4f}")

In [ ]:
# Save best model
best_model.save("best_lstm_manual.h5")

In [ ]:
# Step 1: Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from itertools import product
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Step 2: Load & Clean Data
url = 'https://raw.githubusercontent.com/SankarSivan/Stock-Price-Prediction-Apple/main/AAPL.csv'
df = pd.read_csv(url, parse_dates=["Date"], index_col="Date")
df = df[['Adj Close']].rename(columns={"Adj Close": "adj_close"})
df.fillna(method="ffill", inplace=True)
df.head()

In [ ]:
# Step 3: Create Sequences for Time Series Forecasting
SEQ_LEN = 60
def create_sequences(data, gap=1):
    X, y = [], []
    for i in range(len(data) - SEQ_LEN - gap + 1):
        X.append(data[i:i+SEQ_LEN])
        y.append(data[i+SEQ_LEN+gap-1])
    return np.array(X), np.array(y)

scaler = MinMaxScaler()
scaled = scaler.fit_transform(df)

X, y = create_sequences(scaled, gap=1)
X = X.reshape((X.shape[0], X.shape[1], 1))  # (samples, timesteps, features)

split = int(0.8 * len(X))
X_train, X_val = X[:split], X[split:]
y_train, y_val = y[:split], y[split:]

In [ ]:
# Step 4: Train SimpleRNN
def build_rnn():
    model = Sequential([
        SimpleRNN(50, input_shape=(SEQ_LEN, 1)),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

rnn_model = build_rnn()
es = EarlyStopping(patience=5, restore_best_weights=True)
rnn_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val), callbacks=[es])


In [ ]:
# Step 5: Train LSTM (basic version)
def build_lstm(units=50, lr=0.001):
    model = Sequential([
        LSTM(units, input_shape=(SEQ_LEN, 1)),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss='mse')
    return model

lstm_model = build_lstm()
lstm_model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_val, y_val), callbacks=[es])


In [ ]:
# Step 6: Manual Hyperparameter Tuning for LSTM
param_grid = {
    "units": [32, 64],
    "lr": [0.001, 0.005],
    "epochs": [10, 20]
}
results = []
best_rmse = float("inf")
best_model = None

for units, lr, ep in product(*param_grid.values()):
    print(f"Training: units={units}, lr={lr}, epochs={ep}")
    model = build_lstm(units, lr)
    model.fit(X_train, y_train, epochs=ep, batch_size=32, validation_data=(X_val, y_val), callbacks=[es], verbose=0)
    pred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, pred))
    results.append((units, lr, ep, rmse))

    if rmse < best_rmse:
        best_rmse = rmse
        best_model = model
        best_params = (units, lr, ep)

print("Best:", best_params, "RMSE:", best_rmse)
best_model.save("best_lstm_manual.h5")


In [ ]:
# Step 7: Evaluate and Plot
def inverse_scale(y_scaled):
    y_pad = np.hstack([y_scaled, np.zeros((len(y_scaled), 1))])
    return scaler.inverse_transform(y_pad)[:, 0]

val_pred = best_model.predict(X_val)
val_true_inv = inverse_scale(y_val.reshape(-1, 1))
val_pred_inv = inverse_scale(val_pred)

plt.figure(figsize=(12,5))
plt.plot(val_true_inv, label="Actual Price")
plt.plot(val_pred_inv, label="Predicted Price")
plt.title("Prediction vs Actual")
plt.legend()
plt.show()

mae = mean_absolute_error(val_true_inv, val_pred_inv)
rmse = np.sqrt(mean_squared_error(val_true_inv, val_pred_inv))
print(f"MAE: {mae:.2f} | RMSE: {rmse:.2f}")


In [ ]:
# Step 7: Evaluate and Plot
def inverse_scale(y_scaled):
    y_pad = np.hstack([y_scaled, np.zeros((len(y_scaled), 1))])
    return scaler.inverse_transform(y_pad)[:, 0]

val_pred = best_model.predict(X_val)
val_true_inv = inverse_scale(y_val.reshape(-1, 1))
val_pred_inv = inverse_scale(val_pred)

plt.figure(figsize=(12,5))
plt.plot(val_true_inv, label="Actual Price")
plt.plot(val_pred_inv, label="Predicted Price")
plt.title("Prediction vs Actual")
plt.legend()
plt.show()

mae = mean_absolute_error(val_true_inv, val_pred_inv)
rmse = np.sqrt(mean_squared_error(val_true_inv, val_pred_inv))
print(f"MAE: {mae:.2f} | RMSE: {rmse:.2f}")

# Task
Generate Python code to forecast future values using a trained LSTM model, visualize the forecast, and present the results.

## Prepare the last sequence

### Subtask:
Get the last `SEQ_LEN` data points from your scaled data to use as the initial input for forecasting.


**Reasoning**:
Get the last SEQ_LEN data points and reshape it for forecasting.



In [ ]:
last_sequence = scaled[-SEQ_LEN:].reshape(1, SEQ_LEN, 1)

## Predict future values

### Subtask:
Use a loop to predict the next data point using the current sequence, and then update the sequence by removing the oldest data point and adding the newly predicted one. Repeat this for the desired number of future days.


**Reasoning**:
Implement the forecasting loop to predict future values using the trained LSTM model.



In [ ]:
future_days = 30
future_predictions_scaled = []
current_sequence = last_sequence.copy()

for _ in range(future_days):
    next_day_prediction_scaled = best_model.predict(current_sequence, verbose=0)
    future_predictions_scaled.append(next_day_prediction_scaled[0, 0])
    current_sequence = np.append(current_sequence[:, 1:, :], next_day_prediction_scaled.reshape(1, 1, 1), axis=1)


## Inverse scale the predictions

### Subtask:
Transform the scaled future predictions back to the original price scale.


**Reasoning**:
Convert the scaled future predictions to the original price scale using the fitted scaler.



In [ ]:
future_predictions_scaled_np = np.array(future_predictions_scaled)
future_predictions_scaled_reshaped = np.hstack([future_predictions_scaled_np.reshape(-1, 1), np.zeros((len(future_predictions_scaled_np), scaled.shape[1]-1))])
future_predictions = scaler.inverse_transform(future_predictions_scaled_reshaped)[:, 0]

## Visualize the forecast

### Subtask:
Visualize the forecast by plotting the historical data and the forecasted future values.


**Reasoning**:
Visualize the historical and forecasted stock prices.



In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['adj_close'], label='Historical Price')
forecast_start_index = len(df) - len(future_predictions)
forecast_dates = pd.date_range(start=df.index[-1], periods=len(future_predictions) + 1)[1:]
plt.plot(forecast_dates, future_predictions, label='Forecasted Price')
plt.title('Stock Price Forecast')
plt.xlabel('Date')
plt.ylabel('Adjusted Close Price')
plt.legend()
plt.show()

## Summary:

### Data Analysis Key Findings

*   The last `SEQ_LEN` data points from the scaled data were successfully extracted and reshaped into a (1, `SEQ_LEN`, 1) NumPy array for use as the initial forecasting input.
*   The LSTM model was used to predict future stock prices for 30 days by iteratively predicting the next data point and updating the input sequence.
*   The scaled future predictions were successfully inverse transformed back to the original price scale.
*   A plot was generated showing the historical adjusted close prices and the forecasted future adjusted close prices, providing a visual representation of the model's predictions.

### Insights or Next Steps

*   Evaluate the accuracy of the forecast using appropriate metrics (e.g., RMSE, MAE) by comparing the predictions to actual future stock prices once available.
*   Explore different LSTM model architectures, hyperparameters, or incorporate additional features (e.g., trading volume, news sentiment) to potentially improve forecasting performance.
